<a href="https://colab.research.google.com/github/angelfox4/Pytorch_Introduction/blob/main/softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

## 소프트맥스 회귀 구현하기(low-level)

In [ ]:
# 훈련데이터 선언
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train=torch.FloatTensor(x_train) #8개의 샘플과 4개의 특성
y_train=torch.LongTensor(y_train) #각 샘플에 대한 레이블, 총 3개의 클래스를 가짐

In [ ]:
#y_train one-hot encoding
y_one_hot=torch.zeros(8,3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [ ]:
#모델 초기화
W=torch.zeros((4,3), requires_grad=True)
b=torch.zeros(1, requires_grad=True)
#optimizer 설정
optimizer=optim.SGD([W,b], lr=0.1)

In [ ]:
nb_epochs=1000
for epoch in range(nb_epochs+1):
  #가설
  hypothesis=F.softmax(x_train.matmul(W)+b, dim=1)

  #비용 함수
  cost=(y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()

  #cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## 소프트맥스 회귀 구현(high-level)
F.cross_entropy()를 사용, 주의할 점은 함수가 소프트맥스를 포함하므로 가설 부분이 필요가 없어짐

In [ ]:
#모델 초기화
W=torch.zeros((4,3), requires_grad=True)
b=torch.zeros(1, requires_grad=True)
#optimizer 설정
optimizer=optim.SGD([W,b], lr=0.1)

nb_epochs=1000
for epoch in range(nb_epochs+1):
  #cost 계산
  z=x_train.matmul(W)+b
  cost=F.cross_entropy(z, y_train)

  #cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## nn.Module로 소프트맥스 구현

In [ ]:
#모델 선언 및 초기화, 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3
model=nn.Linear(4,3)

#원래 부분
#W=torch.zeros((4,3), requires_grad=True)
#b=torch.zeros(1, requires_grad=True)

<generator object Module.parameters at 0x7fcb6cc562b0>

In [ ]:
#optimizer 설정
optimizer=optim.SGD(model.parameters(), lr=0.01)

nb_epochs=1000
for epoch in range(nb_epochs+1):
  #cost 계산
  prediction=model(x_train) #원래 부분 : z=x_train.matmul(W)+b
  cost=F.cross_entropy(prediction, y_train)

  #cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.863129
Epoch  200/1000 Cost: 0.760346
Epoch  300/1000 Cost: 0.707352
Epoch  400/1000 Cost: 0.669059
Epoch  500/1000 Cost: 0.639046
Epoch  600/1000 Cost: 0.614619
Epoch  700/1000 Cost: 0.594219
Epoch  800/1000 Cost: 0.576826
Epoch  900/1000 Cost: 0.561739
Epoch 1000/1000 Cost: 0.548455


## 클래스로 소프트맥스 구현

In [ ]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(4,3) #output이 3!

  def forward(self, x):
    return self.linear(x)

In [ ]:
model=SoftmaxClassifierModel()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428281
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818
